In [1]:
example = """px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}"""

In [2]:
with open('./data/Day 19/input.txt') as inputFile:
    data = inputFile.read()

In [3]:
import re

In [4]:
def parse_input(inputstr):
    rules = {}
    parts = []
    rulesection = True
    for line in inputstr.splitlines():
        if line == "":
            rulesection = False
            continue
        if rulesection:
            identifier, rule = line.strip("}").split("{")
            rules[identifier] = re.split(r":|,", rule)
        else:
            cleaned_line = line.strip('{}')
            parts.append(
                {i.split("=")[0]: int(i.split("=")[1]) for i in cleaned_line.split(",")}
            )
    return rules, parts

In [5]:
def test(rules, i, x, m, a, s):
    if eval(rules[i]):
        return rules[i+1], i+1
    else:
        return rules[i+2], i+2

In [6]:
def run_part(ruleset, part):
    current_key = 'in'
    next_action, index = test(ruleset['in'], 0, **part)
    while next_action not in ['A', 'R']:
        if next_action in ruleset.keys():
            current_key = next_action
            next_action, index = test(ruleset[next_action], 0, **part)
        else:
            next_action, index = test(ruleset[current_key], index, **part)
    return next_action

# Part 1

In [7]:
example_rules, example_parts = parse_input(example)
example_score = 0
for part in example_parts:
    part['result'] = run_part(example_rules, part)
    if part['result'] == 'A':
        example_score += part['x'] + part['m'] + part['a'] + part['s']
example_score

19114

In [8]:
rules, parts = parse_input(data)
score = 0
for part in parts:
    part["result"] = run_part(rules, part)
    if part['result'] == 'A':
        score += part['x'] + part['m'] + part['a'] + part['s']
score

325952

# Part 2

In [9]:
def get_condition_tree(rule, i):
    tree = {'condition': rule[i], True: rule[i + 1], False: rule[i + 2]}
    return tree, i

In [10]:
def create_tree(rule, rules):
    if isinstance(rule[1], list):
        rule[1] = create_tree(rule[1], rules)
    if isinstance(rule[2], list):
        rule[2] = create_tree(rule[2], rules)
    if rule[1] not in ['A', 'R'] and not isinstance(rule[1], list):
        rule[1] = create_tree(rules[rule[1]], rules)
    if rule[2] not in ['A', 'R'] and not isinstance(rule[2], list):
        rule[2] = create_tree(rules[rule[2]], rules)
    return rule

In [11]:
def cut_list(input_list):
    if len(input_list) > 3:
        input_list = input_list[:2] + [cut_list(input_list[2:])]
    return input_list

In [12]:
example_rules, _ = parse_input(example)
example_rules_2 = {}
for key, value in example_rules.items():
    value = cut_list(value)
    example_rules_2[key] = value
example_tree = create_tree(example_rules_2["in"], example_rules_2)

In [13]:
rules_2 = {}
for key, value in rules.items():
    value = cut_list(value)
    rules_2[key] = value
tree = create_tree(rules_2["in"], rules_2)

In [47]:
def get_condition(condition):
    if '<' in condition:
        char, value = condition.split('<')
        return {True: {char: range(1, int(value))}, False: {char: range(int(value), 4000)}}
    elif '>' in condition:
        char, value = condition.split('>')
        return {True: {char: range( int(value)+1, 4000)}, False: {char: range(1, int(value)+1)}}

In [48]:
def merge_dictionaries(dict1, dict2):
    for key, value in dict2.items():
        if key in dict1.keys():
            dict1[key] = range(max(dict1[key].start, value.start), min(dict1[key].stop, value.stop))
        else:
            dict1[key] = value
    return dict1

In [58]:
import math

def traverse_tree(tree, current_parents, score):
    root_condition = get_condition(tree[0])
    if isinstance(tree[1], list):
        truth_result, score = traverse_tree(tree[1], merge_dictionaries(current_parents.copy(),root_condition[True]), score)
    else:
        if tree[1] == 'A':
            results = merge_dictionaries(current_parents.copy(),root_condition[True])
            temp_values = []
            for i in 'xmas':
                if i not in results:
                    temp_values.append(4000)
                else:
                    temp_values.append(len(results[i]))
            score += math.prod(temp_values)
        truth_result = tree[1]
    if isinstance(tree[2], list):
        false_result, score = traverse_tree(tree[2], merge_dictionaries(current_parents.copy(),root_condition[False]), score)
    else:
        if tree[2] == 'A':
            results = merge_dictionaries(current_parents.copy(),root_condition[False])
            temp_values = []
            for i in 'xmas':
                if i not in results:
                    temp_values.append(4000)
                else:
                    temp_values.append(len(results[i]))
            score += math.prod(temp_values)
        false_result = tree[2]
    conditions = {"condition": root_condition, True: truth_result, False: false_result}
    return conditions, score

In [61]:
score = 0
_, example_score = traverse_tree(example_tree, {}, score)
example_score

167409079868000

In [62]:
score = 0
_, score = traverse_tree(tree, {}, score)
score

125744206494820